In [1]:
%pip install chatterbot
%pip install chatterbot_corpus

     |████████████████████████████████| 71kB 2.8MB/s 
     |████████████████████████████████| 1.3MB 8.5MB/s 
  Found existing installation: SQLAlchemy 1.4.18
    Uninstalling SQLAlchemy-1.4.18:
      Successfully uninstalled SQLAlchemy-1.4.18
     |████████████████████████████████| 122kB 4.2MB/s 


In [2]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
print("Required modules are successfully imported....")

Required modules are successfully imported....


In [3]:
my_bot = ChatBot(name='MyBot', read_only=True,
                 logic_adapters=['chatterbot.logic.MathematicalEvaluation',
                                 'chatterbot.logic.BestMatch'])

In [4]:
talk1 = ['hi there!',
              'hi!',
              'how do you do?',
              'Cool as always'
              'how are you?',
              'i\'m cool.',
              'fine, you?',
              'always cool.',
              'i\'m ok',
              'glad to hear that.',
              'i\'m fine',
              'glad to hear that.',
              'i feel awesome',
              'excellent, glad to hear that.',
              'good',
              'Thats awesome to hear'
              'not so good',
              'sorry to hear that.',
              'I need your assistance regarding my order',
              'Please, Provide me with your order id',
              'I have a complaint.',
              'Please elaborate, your concern',
              'How long it will take to receive an order ?',
              'An order takes 3-5 Business days to get delivered.',
              'The product delivered is damaged',
              'Oh! we are really sorry for! please provide your order id we will replace it for you',
              'Okay Thanks',
              'No Problem! Have a Good Day!'
              'what\'s your name?',
              'i\'m your bot. Ask me a math question, please.']

In [5]:
talk2 = ['pythagorean theorem',
               'a squared plus b squared equals c squared.']

In [6]:
list_trainer = ListTrainer(my_bot)
for item in (talk1, talk2):
    list_trainer.train(item)
print("Training of chatbot is completed")

List Trainer: [####################] 100%
List Trainer: [####################] 100%
Training of chatbot is completed


In [7]:
print(my_bot.get_response("how are you"))

Cool as alwayshow are you?


In [8]:
name=input("Enter Your Name: ")
print("Hey welcome " + name + "! Let's chat!!!")
while True:
    request=input(name+':')
    if request=='Bye' or request =='bye':
        print('Bot: Ok Bye. See you then!!')
        break
    else:
        response=my_bot.get_response(request)
        print('Bot:',response)

Enter Your Name: Sona
Hey welcome Sona! Let's chat!!!
Sona:Hi
Bot: how do you do?
Sona:Fine
Bot: glad to hear that.
Sona:Bye
Bot: Ok Bye. See you then!!


In [9]:
from chatterbot.trainers import ChatterBotCorpusTrainer
corpus_trainer = ChatterBotCorpusTrainer(my_bot)
corpus_trainer.train('chatterbot.corpus.english')

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [10]:
print(my_bot.get_response("tell me a joke"))

Did you hear the one about the mountain goats in the andes? It was "ba a a a a a d".


In [11]:
import numpy as np
import random
import json
import nltk
from nltk.stem.porter import PorterStemmer

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1
    return bag

In [17]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

In [ ]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [ ]:
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [ ]:
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags),"tags")
print(len(all_words),"words")

In [ ]:
print(xy)

In [ ]:
print(tags)

In [ ]:
print(all_words)

In [ ]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
   bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
num_epochs = 1000
batch_size = 8  
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

In [ ]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [ ]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        outputs = model(words)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')

In [ ]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'Training complete. File saved to {FILE}')

In [ ]:
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model.load_state_dict(model_state)
model.eval()

In [ ]:
bot_name = "Doremon"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")